In [1]:
# Importing Necessary Libraries
import random
import json
import pickle

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD


import numpy as np

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ghkch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ghkch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ghkch\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#Reading the JSON file:
intents = json.loads(open("intentsUpdated.json").read())

# Initializing Lists:

words = []
classes = []
documents = []

# Defining Ignored Characters
ignore_letters = ["?", "!", ".", ","]

In [3]:
#Looping through intents and patterns

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # Tokenization and Data Extraction
        word_list = nltk.word_tokenize(pattern)
        # Updating Lists: 
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        
        # Updating Classes List:

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# Lemmatization:
words = [lemmatizer.lemmatize(word)
         for word in words if word not in ignore_letters]

In [4]:
# Sorting and Removing Duplicates from the 'words' list

words = sorted(set(words))

# Sorting and Removing Duplicates from the 'classes' list
classes = sorted(set(classes))

In [5]:
# Saving processed data using 'pickle.dump':
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb')) 
# 'wb' signifies that files should be opened in binary write mode for writing serialized data

#Initializing 'dataset' and 'template variables'
dataset = []
template = [0]*len(classes)

In [6]:
# Iterating through each document
for document in documents:
    # bag - used for bag-of-words representation for each document
    bag = []
    # Extracting word patterns and lemmatizing:
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower())
                     for word in word_patterns]
    # Creating bag-of-words representation:
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # Creating output row:
    output_row = list(template)
    output_row[classes.index(document[1])] = 1
    # Appending to the dataset:
    dataset.append([bag, output_row])

In [7]:
# Shuffling the dataset to prevent any inherent order or bias in the data from affecting the training process.
random.shuffle(dataset)

# Extracting the input (features) and output (target) data
train_x = [data[0] for data in dataset]  # Features
train_y = [data[1] for data in dataset]  # Target labels

In [8]:
# Converting features into NumPy arrays individually
train_x = np.array(train_x)
# Converting target labels into NumPy arrays individually
train_y = np.array(train_y)

In [9]:
# Scale feature data
scaler = StandardScaler()
train_x_scaled = scaler.fit_transform(train_x)

In [10]:
# Define the model
model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),),
                activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [11]:
# Compile the model
sgd = SGD(learning_rate=0.01,
          momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y,
                 epochs=200, batch_size=5, verbose=1)

Epoch 1/200
144/144 [==============================] - 1s 3ms/step - loss: 4.8735 - accuracy: 0.0070
Epoch 2/200
144/144 [==============================] - 0s 3ms/step - loss: 4.8360 - accuracy: 0.0265
Epoch 3/200
144/144 [==============================] - 0s 3ms/step - loss: 4.8004 - accuracy: 0.0265
Epoch 4/200
144/144 [==============================] - 0s 3ms/step - loss: 4.7571 - accuracy: 0.0293
Epoch 5/200
144/144 [==============================] - 0s 3ms/step - loss: 4.7050 - accuracy: 0.0335
Epoch 6/200
144/144 [==============================] - 0s 3ms/step - loss: 4.6380 - accuracy: 0.0391
Epoch 7/200
144/144 [==============================] - 0s 3ms/step - loss: 4.5398 - accuracy: 0.0573
Epoch 8/200
144/144 [==============================] - 0s 3ms/step - loss: 4.4250 - accuracy: 0.0782
Epoch 9/200
144/144 [==============================] - 0s 3ms/step - loss: 4.2523 - accuracy: 0.1075
Epoch 10/200
144/144 [==============================] - 0s 3ms/step - loss: 4.0956 - accura

144/144 [==============================] - 1s 4ms/step - loss: 0.7342 - accuracy: 0.6983
Epoch 161/200
144/144 [==============================] - 1s 3ms/step - loss: 0.7418 - accuracy: 0.6760
Epoch 162/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7404 - accuracy: 0.7011
Epoch 163/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7375 - accuracy: 0.6830
Epoch 164/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7288 - accuracy: 0.6858
Epoch 165/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7407 - accuracy: 0.6844
Epoch 166/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7198 - accuracy: 0.6927
Epoch 167/200
144/144 [==============================] - 1s 3ms/step - loss: 0.7174 - accuracy: 0.6816
Epoch 168/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7238 - accuracy: 0.6941
Epoch 169/200
144/144 [==============================] - 1s 4ms/step - loss: 0.7141 - a

In [12]:
model.save("chatbot_model.h5", hist)
print("Done!")

Done!


C:\Users\ghkch\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
